# mysql 과 python 연동

1. 라이브러리 설치
2. sql 서버의 정보를 등록
3. sql 서버에 접속
4. sql 쿼리문을 이용하여 데이터를 CRUD 작업


In [1]:
# pymysql (라이브러리) 설치

!pip install pymysql

     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------------------- 43.7/43.7 kB ? eta 0:00:00


In [2]:
import pymysql

In [3]:
# sql 서버에 정보를 입력 

_db = pymysql.connect(
    user = 'root',
    password = '1234',
    host = 'localhost',
    db = 'ubion7',
    port = 3306
)

In [4]:
# cursor 변수 생성
# sql 쿼리문을 실행 시키기 위한 필수변수
# DictCursor : sql쿼리문이 실행되고 나온 데이터를 딕셔너리 형태로 받는다. 의미 
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [12]:
# """""" : 줄이 바뀌어도 이 사이에는 다 string
sql = """
    select
    *
    from
    user_info
"""

cursor.execute(sql)

4

In [13]:
result = cursor.fetchall()

In [14]:
import pandas as pd

In [15]:
pd.DataFrame(result)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01012312312,2
2,test3,1111,test3,010898982323,2
3,test4,1212,test4,01011111111,1


In [16]:
sql = """
    select
    *
    from
    example
"""

cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df

,Name,Gender,Phone
0,A,male,010-1234-5678
1,B,female,010-1234-5679
2,C,male,010-1234-5680
3,D,female,010-1234-5681
4,E,male,010-1234-5682
5,F,female,010-1234-5683
6,G,male,010-1234-5684
7,H,female,010-1234-5685
8,I,male,010-1234-5686
9,J,female,010-1234-5687


In [18]:
# user_info 에 insert 해보자

sql = """
    insert into
    user_info
    values
    ("test5", "1234", "test5", "01011118888", 1)
"""

cursor.execute(sql) # 실제 db에 적용되기 전 -> commit 까지 해야

1

In [19]:
_db.commit() # 이거하고 Workbench에서 Ctrl + Enter 로 selct* from - 실행하면 반영 완료. 

### sql 함수화

1. 첫 번째 함수
    - 매개변수 1개 : sql 쿼리문 ->  select문
    - sql 쿼리문을 실행하고 결과를 dataframe으로 변경하여 리턴
2. 두 번째 함수 
    - 매개변수 1개 : sql 쿼리문 -> 결과값 없는 쿼리문
    - sql 쿼리문을 실행하고 데이터베이스를 변경 
3. 세 번째 함수
    - 1번 함수 & 2번 함수를 하나의 함수에서 실행이 될 수 있도록 생성

In [33]:
# 1st function : 결과값 있는 쿼리문 

def yes_result(sql) : 
    
    cursor.execute(sql)
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    return df


In [34]:
# 1st test

sql = """
    select
    *
    from
    user_info
"""

yes_result(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01012312312,2
2,test3,1111,test3,010898982323,2
3,test4,1212,test4,01011111111,1
4,test5,1234,test5,01011118888,1


In [35]:
# 2nd function : 결과값 없는 쿼리문

def no_result(sql) :

    cursor.execute(sql)
    _db.commit()


    


In [42]:
# 2nd test

_sql = """
    insert into
    user_info
    values 
    ("test8", "2222", "zino" ,"01057390603", 1)
"""

no_result(_sql)

In [76]:
#Moon - 2nd_function

def _sql(sql, values) :

    cursor.execute(sql, values)
    _db.commit()


In [77]:
#Moon

sql = """
    insert into
    user_info
    values
    (%s, %s, %s, %s, %s)
"""

values = ["test13", "1223", "moon", "01098982323", 2]

_sql(sql, values)



In [47]:
# 3rd function : 들어오는 쿼리문 판단 whether 1st or 2nd

def whatever(sql) :

    cursor.execute(sql)

    if sql[0:5] == 'select' :
        result = cursor.fetchall()
        df = pd.DataFrame(result)  
        return df
    else : 
        _db.commit()
    


In [57]:
sql3 = """
    insert into
    user_info
    values 
    ("test13", "2222", "zino" ,"01057390603", 1)
"""

whatever(sql3)

In [67]:
# Moon_ 3rd.function

sql4 = """
    select
    *
    from
    user_info 
"""
sql4.replace('\n', '').strip().startswith('select')

True

In [81]:
# Moon_ 3rd.function

def sql_query(sql, values = []) :
    # if (sql.find("select") != -1) & (sql.find('select') < 10) :
    if (sql.replace('\n', '').strip().startswith('select')) : 
        cursor.execute(sql, values)
        result = cursor.fetchall()
        result = pd.DataFrame(result)
    else : 
        cursor.execute(sql, values)
        _db.commit()
        result = "Query OK"

    return result
        

In [82]:
sql = """
    delete
    from
    user_info
    where
    user_id = %s
"""

values = ['test10']

sql_query(sql, values)

'Query OK'

In [144]:
# 클래스 로 모듈화 후 test
# Moon

import mod_sql

import imp
imp.reload(mod_sql)


<module 'mod_sql' from 'c:\\Users\\zino0\\Documents\\GitHub\\Practice_ubion\\Code_Moon\\230417\\mod_sql\\__init__.py'>

In [145]:
test = mod_sql.Database(
    _host = 'localhost',
    _user = 'root',
    _pass = '1234',
    _db = 'ubion7',
    _port = 3306
    )

In [105]:
_sql0 = """
    select
    *
    from
    user_info
"""



In [98]:
test.sql_query(_sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test10,2222,zono,01052340603,2
2,test11,2222,zino,01057390603,1
3,test12,2222,zino,01057390603,1
4,test13,1223,moon,01098982323,2
5,test2,1234,test2,01012312312,2
6,test3,1111,test3,010898982323,2
7,test4,1212,test4,01011111111,1
8,test5,1234,test5,01011118888,1
9,test8,2222,zino,01057390603,1


In [99]:
test.db

In [101]:
test2 = mod_sql.Database(
    '172.16.11.148',
    'ubion',
    '1234',
    'ubion7',
    3306
)

test2.sql_query(_sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,0101111111,2
2,test4,1234,test4,01043432432,1
3,test5,1234,test5,01011118888,1
4,test6,1234,test6,01011111111,2
5,test7,1234,test7,01030423423,2


In [121]:
_sql = """
    insert into
    user_info
    values
    (%s,%s,%s,%s,%s)
"""

val = ['hungry', '8282', 'youngsang', '01013131313', 2]

test2.sql_query(_sql, val)

'Query OK'

In [116]:
# 컬럼 select할때, as '별칭' 부르면 , 
    # 보여줄때만 컬럼이름 별칭으로 바꿔서 보여줌
# query 문 안에 , limit 3 추가하면 head(3) 처럼 3개만 보여줌
# 컬럼이름에는 물결키 ``로 묶어주면 좋아
    # 컬럼이름 등에 공백있으면 공백기준으로 컬럼이름을 불러오려고해
    # 공백있을 때, ``로 묶어서 한번에 컬럼이름을 인식하게 만들어주자

sql = """
    select
    `Name` as '이름',
    `Phone` as '휴대폰번호'
    from
    example
    where
    Gender = %s
    limit 3
"""
values = ['male']
test.sql_query(sql, values)

,이름,휴대폰번호
0,A,010-1234-5678
1,C,010-1234-5680
2,E,010-1234-5682


In [132]:
# sql 작업이 끝나면 접속을 끊어야 해 

test.sql_close()

In [124]:
# corona 

sql = """
    select
    `createDt` as '등록일시',
    `deathCnt` as '사망자',
    `decideCnt` as '감염자'
    from
    corona
"""

test.sql_query(sql)

,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


### 위 셀 결과값을 pandas로 불러서 같은 형태로 만들어보자

In [136]:
df = pd.read_csv('../csv/corona.csv')
df.columns

Index(['Unnamed: 0', 'createDt', 'deathCnt', 'decideCnt', 'seq', 'stateDt',
       'stateTime', 'updateDt', 'accExamCnt', 'accDefRate'],
      dtype='object')

In [137]:
df.drop(['Unnamed: 0', 'seq', 'stateDt',
       'stateTime', 'updateDt', 'accExamCnt', 'accDefRate'], axis = 1, inplace = True)
df.head(2)

,createDt,deathCnt,decideCnt
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842


In [142]:
df.columns = ['등록일시', '사망자', '감염자']

df

,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


In [146]:
# Sales Records 

sql = """
    select
    `Country` as '국가',
    `Units Sold` as '판매개수',
    `Unit Price` as '가격',
    `Unit Cost` as '원가',
    (`Units Sold` * `Unit Price`) as '총수익',
    (`Units Sold` * `Unit Cost`) as '총비용',
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from
    `Sales Records`
"""

test.sql_query(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


### Pandas 로도 위 셀 형태로 아웃풋 만들어보자

In [151]:
df = pd.read_csv('../csv/Sales Records.csv')
df.columns

Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'Units Sold', 'Unit Price',
       'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit'],
      dtype='object')

In [152]:
df.drop(['Region', 'Item Type', 'Sales Channel', 'Order Priority',\
         'Order Date', 'Order ID', 'Ship Date'], axis = 1, inplace = True)

In [154]:
df.columns = ['국가', '판매개수', '가격', '원가', '총수익', '총비용', '총이윤']

In [155]:
df

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20
